In [4]:
import os
import pandas as pd

data_path = r"C:\Health-care"

train_labels = pd.read_csv(os.path.join(data_path, "Train-1542865627584.csv"))
train_inp = pd.read_csv(os.path.join(data_path, "Train_Inpatientdata-1542865627584.csv"))
train_out = pd.read_csv(os.path.join(data_path, "Train_Outpatientdata-1542865627584.csv"))
train_bene = pd.read_csv(os.path.join(data_path, "Train_Beneficiarydata-1542865627584.csv"))

test_labels = pd.read_csv(os.path.join(data_path, "Test-1542969243754.csv"))
test_inp = pd.read_csv(os.path.join(data_path, "Test_Inpatientdata-1542969243754.csv"))
test_out = pd.read_csv(os.path.join(data_path, "Test_Outpatientdata-1542969243754.csv"))
test_bene = pd.read_csv(os.path.join(data_path, "Test_Beneficiarydata-1542969243754.csv"))

print("train_labels:", train_labels.shape)
print("train_inp:", train_inp.shape)
print("train_out:", train_out.shape)
print("train_bene:", train_bene.shape)
print("test_labels:", test_labels.shape)
print("test_inp:", test_inp.shape)
print("test_out:", test_out.shape)
print("test_bene:", test_bene.shape)

train_labels.head()



train_labels: (5410, 2)
train_inp: (40474, 30)
train_out: (517737, 27)
train_bene: (138556, 25)
test_labels: (1353, 1)
test_inp: (9551, 30)
test_out: (125841, 27)
test_bene: (63968, 25)


,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [5]:
train_labels = train_labels.rename(columns={'PotentialFraud': 'Target'})

print(train_labels['Target'].value_counts())
train_labels.head()


Target
No     4904
Yes     506
Name: count, dtype: int64


,Provider,Target
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [15]:
# Inpatient features per provider 
inp_group = train_inp.groupby('Provider')

inp_agg = inp_group.agg({
    'ClaimID': 'count',
    'InscClaimAmtReimbursed': ['sum', 'mean'],
    'AdmissionDt': 'nunique',
    'DischargeDt': 'nunique',
    'AttendingPhysician': 'nunique',
    'OperatingPhysician': 'nunique',
    'OtherPhysician': 'nunique'
})

inp_agg.columns = ['_'.join(col).strip() for col in inp_agg.columns.values]
inp_agg = inp_agg.add_prefix('inp_').reset_index()

#  Outpatient features per provider 
out_group = train_out.groupby('Provider')

out_agg = out_group.agg({
    'ClaimID': 'count',
    'InscClaimAmtReimbursed': ['sum', 'mean'],
    'AttendingPhysician': 'nunique',
    'OperatingPhysician': 'nunique',
    'OtherPhysician': 'nunique'
})

out_agg.columns = ['_'.join(col).strip() for col in out_agg.columns.values]
out_agg = out_agg.add_prefix('out_').reset_index()

print("inp_agg:", inp_agg.shape)
print("out_agg:", out_agg.shape)
inp_agg.head()



inp_agg: (2092, 9)
out_agg: (5012, 7)


,Provider,inp_ClaimID_count,inp_InscClaimAmtReimbursed_sum,inp_InscClaimAmtReimbursed_mean,inp_AdmissionDt_nunique,inp_DischargeDt_nunique,inp_AttendingPhysician_nunique,inp_OperatingPhysician_nunique,inp_OtherPhysician_nunique
0,PRV51001,5,97000,19400.000000,5,5,4,2,1
1,PRV51003,62,573000,9241.935484,57,58,2,2,0
2,PRV51007,3,19000,6333.333333,3,3,2,1,0
3,PRV51008,2,25000,12500.000000,2,2,2,2,0
4,PRV51011,1,5000,5000.000000,1,1,1,0,0


In [16]:
# Provider–beneficiary mapping from both claim tables
inp_prov_bene = train_inp[['Provider', 'BeneID']].drop_duplicates()
out_prov_bene = train_out[['Provider', 'BeneID']].drop_duplicates()

prov_bene = pd.concat([inp_prov_bene, out_prov_bene], axis=0).drop_duplicates()
print("prov_bene:", prov_bene.shape)

# Join with beneficiary table
prov_bene_full = prov_bene.merge(train_bene, on='BeneID', how='left')
print("prov_bene_full:", prov_bene_full.shape)
prov_bene_full.head()


prov_bene: (363300, 2)
prov_bene_full: (363300, 26)


,Provider,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,PRV55912,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,...,1,1,1,2,1,1,36000,3204,60,70
1,PRV55907,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,...,1,1,1,2,1,1,36000,3204,60,70
2,PRV56046,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,...,1,1,1,2,1,1,36000,3204,60,70
3,PRV52405,BENE11011,1914-03-01,NaN,2,2,0,1,360,12,...,1,1,2,2,1,1,5000,1068,250,320
4,PRV56614,BENE11014,1938-04-01,NaN,2,1,Y,45,780,12,...,1,2,1,2,2,2,21260,2136,120,100


In [17]:
# Aggregate beneficiary info per provider
bene_group = prov_bene_full.groupby('Provider')

bene_agg = bene_group.agg({
    'BeneID': 'nunique',
    'Gender': 'nunique',
    'Race': 'nunique',
    'NoOfMonths_PartACov': 'mean',
    'NoOfMonths_PartBCov': 'mean',
    'ChronicCond_Alzheimer': 'mean',
    'ChronicCond_Heartfailure': 'mean',
    'ChronicCond_KidneyDisease': 'mean',
    'ChronicCond_Cancer': 'mean',
    'ChronicCond_ObstrPulmonary': 'mean',
    'ChronicCond_Depression': 'mean',
    'ChronicCond_Diabetes': 'mean',
    'ChronicCond_IschemicHeart': 'mean',
    'ChronicCond_Osteoporasis': 'mean',
    'ChronicCond_rheumatoidarthritis': 'mean',
    'ChronicCond_stroke': 'mean',
    'IPAnnualReimbursementAmt': 'mean',
    'IPAnnualDeductibleAmt': 'mean',
    'OPAnnualReimbursementAmt': 'mean',
    'OPAnnualDeductibleAmt': 'mean'
})

bene_agg.columns = ['bene_' + col for col in bene_agg.columns]
bene_agg = bene_agg.reset_index()

print("bene_agg:", bene_agg.shape)
bene_agg.head()



bene_agg: (5410, 21)


,Provider,bene_BeneID,bene_Gender,bene_Race,bene_NoOfMonths_PartACov,bene_NoOfMonths_PartBCov,bene_ChronicCond_Alzheimer,bene_ChronicCond_Heartfailure,bene_ChronicCond_KidneyDisease,bene_ChronicCond_Cancer,...,bene_ChronicCond_Depression,bene_ChronicCond_Diabetes,bene_ChronicCond_IschemicHeart,bene_ChronicCond_Osteoporasis,bene_ChronicCond_rheumatoidarthritis,bene_ChronicCond_stroke,bene_IPAnnualReimbursementAmt,bene_IPAnnualDeductibleAmt,bene_OPAnnualReimbursementAmt,bene_OPAnnualDeductibleAmt
0,PRV51001,24,2,2,12.000000,12.000000,1.416667,1.250000,1.291667,1.791667,...,1.625000,1.166667,1.083333,1.750000,1.666667,1.791667,18047.916667,890.000000,2537.500000,474.916667
1,PRV51003,117,2,3,11.794872,11.854701,1.623932,1.401709,1.555556,1.914530,...,1.598291,1.256410,1.153846,1.760684,1.726496,1.923077,6814.017094,822.632479,2490.598291,664.529915
2,PRV51004,138,2,3,11.855072,11.956522,1.565217,1.405797,1.659420,1.884058,...,1.565217,1.304348,1.289855,1.688406,1.702899,1.884058,4596.739130,454.144928,2095.144928,600.869565
3,PRV51005,495,2,3,11.830303,11.886869,1.666667,1.468687,1.640404,1.880808,...,1.628283,1.365657,1.298990,1.713131,1.743434,1.921212,3717.232323,398.698990,1798.808081,475.965657
4,PRV51007,58,2,2,11.793103,11.793103,1.637931,1.482759,1.706897,1.896552,...,1.637931,1.379310,1.310345,1.706897,1.724138,1.844828,3109.655172,423.517241,1497.241379,430.689655


In [18]:
# Merge everything into one provider-level table
train_df = train_labels.merge(inp_agg, on='Provider', how='left')
train_df = train_df.merge(out_agg, on='Provider', how='left')
train_df = train_df.merge(bene_agg, on='Provider', how='left')

print(train_df.shape)
train_df.head()


(5410, 36)


,Provider,Target,inp_ClaimID_count,inp_InscClaimAmtReimbursed_sum,inp_InscClaimAmtReimbursed_mean,inp_AdmissionDt_nunique,inp_DischargeDt_nunique,inp_AttendingPhysician_nunique,inp_OperatingPhysician_nunique,inp_OtherPhysician_nunique,...,bene_ChronicCond_Depression,bene_ChronicCond_Diabetes,bene_ChronicCond_IschemicHeart,bene_ChronicCond_Osteoporasis,bene_ChronicCond_rheumatoidarthritis,bene_ChronicCond_stroke,bene_IPAnnualReimbursementAmt,bene_IPAnnualDeductibleAmt,bene_OPAnnualReimbursementAmt,bene_OPAnnualDeductibleAmt
0,PRV51001,No,5.0,97000.0,19400.000000,5.0,5.0,4.0,2.0,1.0,...,1.625000,1.166667,1.083333,1.750000,1.666667,1.791667,18047.916667,890.000000,2537.500000,474.916667
1,PRV51003,Yes,62.0,573000.0,9241.935484,57.0,58.0,2.0,2.0,0.0,...,1.598291,1.256410,1.153846,1.760684,1.726496,1.923077,6814.017094,822.632479,2490.598291,664.529915
2,PRV51004,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.565217,1.304348,1.289855,1.688406,1.702899,1.884058,4596.739130,454.144928,2095.144928,600.869565
3,PRV51005,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.628283,1.365657,1.298990,1.713131,1.743434,1.921212,3717.232323,398.698990,1798.808081,475.965657
4,PRV51007,No,3.0,19000.0,6333.333333,3.0,3.0,2.0,1.0,0.0,...,1.637931,1.379310,1.310345,1.706897,1.724138,1.844828,3109.655172,423.517241,1497.241379,430.689655


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Encode Target: Yes = 1, No = 0
train_df['Target_bin'] = train_df['Target'].map({'Yes': 1, 'No': 0})

# Drop non‑numeric and label columns for X
drop_cols = ['Provider', 'Target', 'Target_bin']
X = train_df.drop(columns=drop_cols)
y = train_df['Target_bin']

# Fill missing numeric values
X = X.fillna(0)


In [22]:
print(train_df.columns.tolist())
print(train_df[['Provider']].head())


['Provider', 'Target', 'inp_ClaimID_count', 'inp_InscClaimAmtReimbursed_sum', 'inp_InscClaimAmtReimbursed_mean', 'inp_AdmissionDt_nunique', 'inp_DischargeDt_nunique', 'inp_AttendingPhysician_nunique', 'inp_OperatingPhysician_nunique', 'inp_OtherPhysician_nunique', 'out_ClaimID_count', 'out_InscClaimAmtReimbursed_sum', 'out_InscClaimAmtReimbursed_mean', 'out_AttendingPhysician_nunique', 'out_OperatingPhysician_nunique', 'out_OtherPhysician_nunique', 'bene_BeneID', 'bene_Gender', 'bene_Race', 'bene_NoOfMonths_PartACov', 'bene_NoOfMonths_PartBCov', 'bene_ChronicCond_Alzheimer', 'bene_ChronicCond_Heartfailure', 'bene_ChronicCond_KidneyDisease', 'bene_ChronicCond_Cancer', 'bene_ChronicCond_ObstrPulmonary', 'bene_ChronicCond_Depression', 'bene_ChronicCond_Diabetes', 'bene_ChronicCond_IschemicHeart', 'bene_ChronicCond_Osteoporasis', 'bene_ChronicCond_rheumatoidarthritis', 'bene_ChronicCond_stroke', 'bene_IPAnnualReimbursementAmt', 'bene_IPAnnualDeductibleAmt', 'bene_OPAnnualReimbursementAmt',

In [23]:
if 'PotentialFraud' in train_df.columns:
    train_df = train_df.rename(columns={'PotentialFraud': 'Target'})

print([c for c in train_df.columns if 'Target' in c])
train_df.head()


['Target', 'Target_bin']


,Provider,Target,inp_ClaimID_count,inp_InscClaimAmtReimbursed_sum,inp_InscClaimAmtReimbursed_mean,inp_AdmissionDt_nunique,inp_DischargeDt_nunique,inp_AttendingPhysician_nunique,inp_OperatingPhysician_nunique,inp_OtherPhysician_nunique,...,bene_ChronicCond_Diabetes,bene_ChronicCond_IschemicHeart,bene_ChronicCond_Osteoporasis,bene_ChronicCond_rheumatoidarthritis,bene_ChronicCond_stroke,bene_IPAnnualReimbursementAmt,bene_IPAnnualDeductibleAmt,bene_OPAnnualReimbursementAmt,bene_OPAnnualDeductibleAmt,Target_bin
0,PRV51001,No,5.0,97000.0,19400.000000,5.0,5.0,4.0,2.0,1.0,...,1.166667,1.083333,1.750000,1.666667,1.791667,18047.916667,890.000000,2537.500000,474.916667,0
1,PRV51003,Yes,62.0,573000.0,9241.935484,57.0,58.0,2.0,2.0,0.0,...,1.256410,1.153846,1.760684,1.726496,1.923077,6814.017094,822.632479,2490.598291,664.529915,1
2,PRV51004,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.304348,1.289855,1.688406,1.702899,1.884058,4596.739130,454.144928,2095.144928,600.869565,0
3,PRV51005,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.365657,1.298990,1.713131,1.743434,1.921212,3717.232323,398.698990,1798.808081,475.965657,1
4,PRV51007,No,3.0,19000.0,6333.333333,3.0,3.0,2.0,1.0,0.0,...,1.379310,1.310345,1.706897,1.724138,1.844828,3109.655172,423.517241,1497.241379,430.689655,0


In [24]:
# 1) create binary target
train_df['Target_bin'] = train_df['Target'].map({'Yes': 1, 'No': 0})

print(train_df['Target'].value_counts())
print(train_df['Target_bin'].value_counts(dropna=False).head())


Target
No     4904
Yes     506
Name: count, dtype: int64
Target_bin
0    4904
1     506
Name: count, dtype: int64


In [25]:
# 2) define features X and label y
drop_cols = ['Provider', 'Target', 'Target_bin']

X = train_df.drop(columns=drop_cols)
y = train_df['Target_bin']

print(X.shape, y.shape)
print(X.dtypes.head())


(5410, 34) (5410,)
inp_ClaimID_count                  float64
inp_InscClaimAmtReimbursed_sum     float64
inp_InscClaimAmtReimbursed_mean    float64
inp_AdmissionDt_nunique            float64
inp_DischargeDt_nunique            float64
dtype: object


In [26]:
# 3) handle missing values
X = X.fillna(0)
print("Any NaN left in X?", X.isna().sum().sum())


Any NaN left in X? 0


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Features and target
drop_cols = ['Provider', 'Target', 'Target_bin']
X = train_df.drop(columns=drop_cols)
y = train_df['Target_bin']

# Fill missing values
X = X.fillna(0)

print(X.shape, y.shape)


(5410, 34) (5410,)


In [28]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_val.shape)
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))


(4328, 34) (1082, 34)
Target_bin
0    0.906423
1    0.093577
Name: proportion, dtype: float64
Target_bin
0    0.906654
1    0.093346
Name: proportion, dtype: float64


In [31]:
log_clf = LogisticRegression(
    max_iter=3000,          # increased
    class_weight='balanced',
    solver='lbfgs'          # default
)
log_clf.fit(X_train, y_train)

y_pred_log = log_clf.predict(X_val)
y_proba_log = log_clf.predict_proba(X_val)[:, 1]

print("Logistic Regression (max_iter=3000)")
print(classification_report(y_val, y_pred_log))
print("ROC-AUC:", roc_auc_score(y_val, y_proba_log))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred_log))


Logistic Regression (max_iter=3000)
              precision    recall  f1-score   support

           0       0.99      0.87      0.92       981
           1       0.41      0.88      0.56       101

    accuracy                           0.87      1082
   macro avg       0.70      0.87      0.74      1082
weighted avg       0.93      0.87      0.89      1082

ROC-AUC: 0.9527053622793471
Confusion matrix:
 [[851 130]
 [ 12  89]]


C:\Users\Manju Bhargavi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Features and target
drop_cols = ['Provider', 'Target', 'Target_bin']
X = train_df.drop(columns=drop_cols)
y = train_df['Target_bin']

X = X.fillna(0)

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_val.shape)

# Random Forest only
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    class_weight='balanced'
)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_val)
y_proba_rf = rf_clf.predict_proba(X_val)[:, 1]

print("Random Forest")
print(classification_report(y_val, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_val, y_proba_rf))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred_rf))


(4328, 34) (1082, 34)
Random Forest
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       981
           1       0.74      0.49      0.59       101

    accuracy                           0.94      1082
   macro avg       0.85      0.73      0.78      1082
weighted avg       0.93      0.94      0.93      1082

ROC-AUC: 0.9442627749013432
Confusion matrix:
 [[964  17]
 [ 52  49]]


In [33]:
import pandas as pd
import numpy as np

feature_importances = pd.Series(
    rf_clf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

feature_importances.head(20)


inp_InscClaimAmtReimbursed_sum     0.125134
inp_ClaimID_count                  0.079091
inp_DischargeDt_nunique            0.071675
inp_AdmissionDt_nunique            0.071450
inp_InscClaimAmtReimbursed_mean    0.063119
out_ClaimID_count                  0.052007
bene_BeneID                        0.051751
out_InscClaimAmtReimbursed_sum     0.045083
inp_OperatingPhysician_nunique     0.038300
inp_AttendingPhysician_nunique     0.027386
bene_IPAnnualReimbursementAmt      0.026109
bene_NoOfMonths_PartBCov           0.024209
bene_OPAnnualDeductibleAmt         0.019121
bene_OPAnnualReimbursementAmt      0.018741
bene_IPAnnualDeductibleAmt         0.018346
bene_ChronicCond_stroke            0.017359
bene_ChronicCond_Diabetes          0.017134
out_InscClaimAmtReimbursed_mean    0.016747
bene_ChronicCond_Alzheimer         0.016109
bene_ChronicCond_Depression        0.016075
dtype: float64